In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as mcolors
from typing import Dict, Any
import names as n

from qdesignoptimizer.utils.plotting import load_data_by_date, plot_optimization_results
from qdesignoptimizer.sim_plot_progress import DataExtractor, OptimizationPlotter
from qdesignoptimizer.sim_plot_progress import OptPltSet
from qdesignoptimizer.utils.names_parameters import (
    param,
    param_capacitance,
    param_nonlin,
)

In [ ]:
import optimization_targets as ot

# Figures for publication

## utils

In [ ]:
# selected colors based on Gothenburg tram line map
colors_got = {'green': '#009639', 
              'blue': '#0066CC', 
              'yellow': '#FFD200', 
              'red': '#E30613', 
              'pink': '#EC008C', 
              'lightblue': '#00B5E2', 
              'purple': '#93328E', 
              'orange': '#F58220', 
              'grey': '#8C8C8C',
              'lightsalmon': '#FF9E80'}

# make a list of colors
color_list = list(colors_got.values())

In [ ]:
def lighten_color(hex_color, factor=0.3):
    """Lighten a hex color by mixing with white"""
    rgb = np.array(mcolors.hex2color(hex_color))
    # Mix with white: new_color = color + factor * (white - color)
    lightened = rgb + factor * (1 - rgb)
    return mcolors.rgb2hex(lightened)

def darken_color(hex_color, factor=0.3):
    """Darken a hex color by mixing with black"""
    rgb = np.array(mcolors.hex2color(hex_color))
    # Mix with black: new_color = color * (1 - factor)
    darkened = rgb * (1 - factor)
    return mcolors.rgb2hex(darkened)

def get_color_shades(hex_color, num_shades=5):
    """
    Generate a list of color shades from dark to light with original color in center.
    
    Parameters:
    -----------
    hex_color : str
        Base hex color (e.g., '#009639')
    num_shades : int
        Number of shades on each side of the original color
        Total colors returned = 2 * num_shades + 1
        
    Returns:
    --------
    list
        List of hex colors ordered from darkest to lightest
        [darkest, ..., darker, ORIGINAL, lighter, ..., lightest]
    """
    shades = []
    
    # Generate darker shades (from darkest to less dark)
    for i in range(num_shades, 0, -1):
        factor = i / (num_shades + 3)  # Avoid pure black by using num_shades+3
        dark_shade = darken_color(hex_color, factor)
        shades.append(dark_shade)
    
    # Add original color in the center
    shades.append(hex_color)
    
    # Generate lighter shades (from less light to lightest)
    for i in range(1, num_shades + 1):
        factor = i / (num_shades + 1)  # Avoid pure white by using num_shades+1
        light_shade = lighten_color(hex_color, factor)
        shades.append(light_shade)
    
    return shades

color_shades_purple = get_color_shades(colors_got['purple'], num_shades=5)
color_shades_red = get_color_shades(colors_got['red'], num_shades=5)
color_shades_blue = get_color_shades(colors_got['blue'], num_shades=5)
color_shades_lightblue = get_color_shades(colors_got['lightblue'], num_shades=4)
color_shades_grey = get_color_shades(colors_got['grey'], num_shades=5)

In [ ]:
def crop_image_by_fraction(image_path, left=0, right=0, top=0, bottom=0, rotate_90=0):
    """
    Crop an image by specified fractions from each edge and optionally rotate.
    
    Parameters:
    -----------
    image_path : str
        Path to the image file
    left : float
        Fraction to crop from left edge (0-1)
    right : float  
        Fraction to crop from right edge (0-1)
    top : float
        Fraction to crop from top edge (0-1)
    bottom : float
        Fraction to crop from bottom edge (0-1)
    rotate_90 : int
        Number of 90-degree rotations (0, 1, 2, 3 or negative values)
        0: no rotation, 1: 90° clockwise, 2: 180°, 3: 270° clockwise, etc.
        
    Returns:
    --------
    numpy.ndarray
        Cropped and rotated image array
    """
    # Read the image
    img = mpimg.imread(image_path)
    
    # Get dimensions
    height, width = img.shape[:2]  # Works for both grayscale and color images
    print(f"Original image size: {width}x{height}")
    
    # Calculate crop boundaries
    left_px = int(width * left)
    right_px = int(width * (1 - right))
    top_px = int(height * top)
    bottom_px = int(height * (1 - bottom))
    
    # Crop the image using array slicing
    # Format: img[top:bottom, left:right] for 2D
    # Format: img[top:bottom, left:right, :] for 3D (color)
    if len(img.shape) == 3:  # Color image
        cropped_img = img[top_px:bottom_px, left_px:right_px, :]
    else:  # Grayscale image
        cropped_img = img[top_px:bottom_px, left_px:right_px]
    
    new_height, new_width = cropped_img.shape[:2]
    print(f"Cropped image size: {new_width}x{new_height}")
    
    # Apply rotation if specified
    if rotate_90 != 0:
        # Normalize rotation to 0-3 range
        rotation_steps = rotate_90 % 4
        if rotation_steps != 0:
            cropped_img = np.rot90(cropped_img, k=rotation_steps)
            final_height, final_width = cropped_img.shape[:2]
            print(f"After {rotate_90 * 90}° rotation: {final_width}x{final_height}")
    
    return cropped_img

In [ ]:
import json
import re
import random
from typing import Dict, Union, Optional

def parse_value_with_unit(value_str: str) -> tuple:
    """Parse a value string like '400um' into (400.0, 'um')"""
    match = re.match(r'^([+-]?\d*\.?\d+)([a-zA-Z]*)$', value_str.strip())
    if match:
        number = float(match.group(1))
        unit = match.group(2)
        return number, unit
    else:
        raise ValueError(f"Cannot parse value: {value_str}")

def format_value_with_unit(number: float, unit: str) -> str:
    """Format a number with unit back to string"""
    if unit == '':
        return str(number)
    else:
        # Handle integer values for cleaner output
        if number == int(number):
            return f"{int(number)}{unit}"
        else:
            return f"{number:.1f}{unit}"

def scramble_parameters(
    json_file_path: str,
    output_file_path: str,
    parameter_bounds: Dict[str, Dict[str, str]],
    seed: Optional[int] = None
) -> Dict[str, str]:
    """
    Scramble specific parameters from a JSON file with explicit bounds.
    
    Parameters:
    -----------
    json_file_path : str
        Path to input JSON file
    output_file_path : str
        Path to save scrambled JSON file
    parameter_bounds : dict
        Dictionary specifying which parameters to scramble and their bounds.
        Format:
        {
            "design_var_width_qubit_1": {"min": "300um", "max": "500um"},
            "design_var_lj_qubit_2": {"min": "8nH", "max": "15nH"},
            "design_var_cl_pos_x_qubit_1": {"min": "-200um", "max": "200um"}
        }
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    dict
        Dictionary with scrambled parameters
        
    Example:
    --------
    bounds = {
        "design_var_width_qubit_1": {"min": "350um", "max": "450um"},
        "design_var_width_qubit_2": {"min": "380um", "max": "420um"},
        "design_var_lj_qubit_1": {"min": "10nH", "max": "14nH"},
        "design_var_lj_qubit_2": {"min": "8nH", "max": "12nH"},
        "design_var_cl_pos_x_qubit_1": {"min": "-150um", "max": "150um"},
        "design_var_cl_pos_y_qubit_1": {"min": "-350um", "max": "-250um"}
    }
    
    scrambled = scramble_parameters("input.json", "output.json", bounds)
    """
    
    if seed is not None:
        random.seed(seed)
    
    # Read original parameters
    with open(json_file_path, 'r') as f:
        original_params = json.load(f)
    
    # Copy original parameters
    scrambled_params = original_params.copy()
    
    # Track what was scrambled
    scrambled_count = 0
    
    # Process each parameter specified in bounds
    for param_name, bounds in parameter_bounds.items():
        
        # Check if parameter exists in original file
        if param_name not in original_params:
            print(f"Warning: Parameter '{param_name}' not found in JSON file")
            continue
        
        original_value = original_params[param_name]
        
        # Parse original value to get unit
        try:
            orig_number, orig_unit = parse_value_with_unit(original_value)
        except ValueError:
            print(f"Warning: Could not parse {param_name}: {original_value}")
            continue
        
        # Parse min and max bounds
        try:
            min_number, min_unit = parse_value_with_unit(bounds["min"])
            max_number, max_unit = parse_value_with_unit(bounds["max"])
        except (ValueError, KeyError) as e:
            print(f"Warning: Invalid bounds for {param_name}: {e}")
            continue
        
        # Check unit consistency
        if min_unit != max_unit:
            print(f"Warning: Unit mismatch in bounds for {param_name}: {min_unit} vs {max_unit}")
            continue
        
        if min_unit != orig_unit:
            print(f"Warning: Bound units ({min_unit}) don't match original unit ({orig_unit}) for {param_name}")
            # Use original unit but convert bounds if possible
            target_unit = orig_unit
        else:
            target_unit = orig_unit
        
        # Ensure min < max
        if min_number >= max_number:
            print(f"Warning: Invalid range for {param_name}: min ({min_number}) >= max ({max_number})")
            continue
        
        # Generate random value
        new_number = random.uniform(min_number, max_number)
        new_value = format_value_with_unit(new_number, target_unit)
        
        # Update parameter
        scrambled_params[param_name] = new_value
        scrambled_count += 1
        print(f"Scrambled {param_name}: {original_value} → {new_value}")
    
    # Save scrambled parameters
    with open(output_file_path, 'w') as f:
        json.dump(scrambled_params, f, indent=4)
    
    print(f"\nScrambled {scrambled_count} parameters")
    print(f"Scrambled parameters saved to: {output_file_path}")
    return scrambled_params

In [ ]:
final_values_after_optimization = {
    "design_var_lj_qubit_1": "14.400163475217242 nH",
    "design_var_width_qubit_1": "822.9178863519965 um",
    "design_var_length_resonator_1": "8084.239839659384 um",
    "design_var_coupl_length_resonator_1_tee": "1086.6975079693889 um",
    "design_var_coupl_length_qubit_1_resonator_1": "439.04984991847476 um"
}

def compute_plus_minus_50_percent(final_values: dict) -> dict:
    """Compute the plus and minus 50 percent variations of the final design variables."""
    variations = {}
    for key, value in final_values.items():
        # Extract the numerical value and unit
        num_value = float(value.split(" ")[0])
        unit = value.split(" ")[1]
        # Compute the variations
        variations[f"{key}_plus_50_percent"] = f"{num_value * 1.5} {unit}"
        variations[f"{key}_minus_50_percent"] = f"{num_value * 0.5} {unit}"
    return variations

compute_plus_minus_50_percent(final_values_after_optimization)

In [ ]:
# for i in range(10):
#     scramble_parameters(
#     json_file_path='design_variables.json',
#     output_file_path=f'design_variables_v{i}.json',
#     parameter_bounds={
#         # 1st initial conditions
#         # "design_var_width_qubit_1": {"min": "100um", "max": "900um"},
#         # "design_var_lj_qubit_1": {"min": "8nH", "max": "18nH"},
#         # "design_var_length_resonator_1": {"min": "5500um", "max": "9500um"},
#         # "design_var_coupl_length_qubit_1_resonator_1": {"min": "10um", "max": "800um"},
#         # "design_var_coupl_length_resonator_1_tee": {"min": "100um", "max": "1300um"},
#         # wider initial conditions (small value, max constraint by geometry)
#        "design_var_width_qubit_1": {"min": "100um", "max": "1100um"},
#        "design_var_lj_qubit_1": {"min": "5nH", "max": "25nH"},
#        "design_var_length_resonator_1": {"min": "4000um", "max": "12000um"},
#        "design_var_coupl_length_qubit_1_resonator_1": {"min": "100um", "max": "1100um"},
#        "design_var_coupl_length_resonator_1_tee": {"min": "100um", "max": "1400um"},

#     },
# )
#to be executed with main_eigenmode_single_qubit_resonator.ipynb

## Fig 3: Single qubit example

In [ ]:
parent_path = r"C:\Users\lukassp\OneDrive - Chalmers\202Q-lab\PROJECTS\QDesignOptimizer\Files_for_paper"
child_path = r"\single_qubit_resonator_fullchi"
child_path_detailed = r'\single_qubit_resonator_g2deltadeltaalpha_10_8'


### Load iamge

In [ ]:
cropped_image = crop_image_by_fraction(parent_path + child_path + r'\2025-08-10_OPT_qubit_resonator_final_sb.png', left=0.1, right=0.1, top=0.0, bottom=0.0, rotate_90=1)


### Load data

In [ ]:
# f_sim_result = [parent_path + child_path + r"\multi_transmon_chip_20250810-175429.npy"]
f_sim_result = [parent_path + child_path_detailed + r"\multi_transmon_chip_20250821-221223.npy"]

In [ ]:
d_sim_result = np.load(f_sim_result[0], allow_pickle=True)[0]
system_target_params = d_sim_result["system_target_params"]
opt_result = d_sim_result['optimization_results']

opt_target_list = ot.get_opt_targets_2qubits_resonator_coupler([1], True,True,True,True,True,False,False)

iteration = 'Iteration'
config_Fig3 = {
    'RES_freq': OptPltSet(n.ITERATION, param(n.RESONATOR_1, n.FREQ), y_label="$f_{res}$", unit="GHz"), 
    'RES_kappa': OptPltSet(n.ITERATION, param(n.RESONATOR_1, n.KAPPA), y_label="$\kappa_{res}$", unit="MHz"),
    'QUB_freq': OptPltSet(n.ITERATION, param(n.QUBIT_1, n.FREQ), y_label="$f_{qb}$", unit="GHz"),
    'QUB_anh': OptPltSet(n.ITERATION, param_nonlin(n.QUBIT_1, n.QUBIT_1), y_label="$\\alpha_{qb}$", unit="MHz"),
    'RES_QUB_chi': OptPltSet(n.ITERATION, param_nonlin(n.RESONATOR_1, n.QUBIT_1), y_label="$\chi_{qb-res}$", unit="MHz"),
    }

print(system_target_params)
print(opt_result[0]['design_variables'])


In [ ]:
data_extractor = DataExtractor([opt_result], system_target_params, opt_target_list)
plotter_Fig3 = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)

### Plot figure with data

In [ ]:
figure_width_inch = 3.35
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, 3.5))
gs = fig.add_gridspec(3, 2, wspace=0.55, hspace=0.15, left=0.16, right=0.99, bottom=0.13, top=0.97)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[2, 0])
ax5 = fig.add_subplot(gs[2, 1])

plotter_Fig3._plot_single_param(ax1, config_Fig3['RES_freq'], y_param=config_Fig3['RES_freq'].y, color=colors_got['blue'], markersize=5)
plotter_Fig3._plot_single_param(ax2, config_Fig3['RES_kappa'], y_param=config_Fig3['RES_kappa'].y, color=colors_got['lightblue'], markersize=5)
plotter_Fig3._plot_single_param(ax3, config_Fig3['QUB_freq'], y_param=config_Fig3['QUB_freq'].y, color=colors_got['purple'], markersize=5)
plotter_Fig3._plot_single_param(ax4, config_Fig3['QUB_anh'], y_param=config_Fig3['QUB_anh'].y, color=colors_got['pink'], markersize=5)
plotter_Fig3._plot_single_param(ax5, config_Fig3['RES_QUB_chi'], y_param=config_Fig3['RES_QUB_chi'].y, color=colors_got['orange'], markersize=5)

ax1.set_xticks([5,10])
ax2.set_xticks([5,10])
ax3.set_xticks([5,10])
ax4.set_xticks([5,10])
ax5.set_xticks([5,10])

ax1.set_xlabel('')
ax2.set_xlabel('')
ax3.set_xlabel('')
ax4.set_xlabel('Iteration')
ax5.set_xlabel('Iteration')

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax3.set_xticklabels([])

ax1.text(0.05,0.1,'(a)', transform=ax0.transAxes)
ax1.text(0.1,0.86,'(b)', transform=ax1.transAxes)
ax2.text(0.04,0.74,'(c)', transform=ax2.transAxes)
ax3.text(0.1,0.86,'(d)', transform=ax3.transAxes)
ax4.text(0.1,0.86,'(e)', transform=ax4.transAxes)
ax5.text(0.1,0.86,'(f)', transform=ax5.transAxes)

ax0.imshow(cropped_image)
ax0.axis('off')

fig.savefig(parent_path + r"\Fig2_OPT_single_qubit_resonator.png", dpi=300)

## Fig 4: Effect of nonlinear model

In [ ]:
iteration = 'Iteration'
config = {
    'RES_QUB_chi': OptPltSet(n.ITERATION, param_nonlin(n.RESONATOR_1, n.QUBIT_1), y_label="$\chi_{qb-res}$", unit="MHz"),
    'QUB_freq': OptPltSet(n.ITERATION, param(n.QUBIT_1, n.FREQ), y_label="$f_{qb}$", unit="GHz"),
    'RES_freq': OptPltSet(n.ITERATION, param(n.RESONATOR_1, n.FREQ), y_label="$f_{res}$", unit="GHz")
    }

### Load data (nonlinear model)

In [ ]:
child_path_simple_chi = r"\single_qubit_resonator_simple"
child_path_sqrtx = r'\single_qubit_resonator_sqrtx_10_8'
child_path_sqrtx_adj05 = r'\single_qubit_resonator_sqrtx_10_8_adj0p5'
child_path_x2 = r'\single_qubit_resonator_x2chi_10_8'
child_path_logx = r'\single_qubit_resonator_logxchi_10_8'
child_path_x3 = r'\single_qubit_resonator_x3chi_10_4'

In [ ]:
f_sim_result = [
    # parent_path + child_path + r"\multi_transmon_chip_20250810-175429.npy", # this is for the "old" chi model
                parent_path + child_path_detailed + r"\multi_transmon_chip_20250821-221223.npy",
                parent_path + child_path_simple_chi + r"\multi_transmon_chip_20250813-100952.npy", 
                parent_path + child_path_sqrtx + r"\multi_transmon_chip_20250818-230313.npy", 
                parent_path + child_path_x2 + r"\multi_transmon_chip_20250818-120022.npy", 
                parent_path + child_path_logx + r"\multi_transmon_chip_20250819-091159.npy", 
                parent_path + child_path_sqrtx_adj05 + r'\multi_transmon_chip_20250820-143805.npy', 
                parent_path + child_path_x3 + r'\multi_transmon_chip_20250821-175639.npy',
               ] 

In [ ]:
opt_plot_Fig4 = []
opt_target_list = ot.get_opt_targets_2qubits_resonator_coupler([1], True,True,True,True,True,False,False)
for i in range(len(f_sim_result)):
    d_sim_result = np.load(f_sim_result[i], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params,opt_target_list)
    plotter = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)
    opt_plot_Fig4.append(plotter)

### Load data (variations)

In [ ]:
# 10_4 narrow initials 
# child_path_initials = r"\single_qubit_resonator_fullchi_10_4_initials"
# f_sim_result_variation = [r"\multi_transmon_chip_20250810-175429.npy", 
#                           r"\multi_transmon_chip_20250811-114202.npy", 
#                           r"\multi_transmon_chip_20250811-123836.npy", 
#                           r"\multi_transmon_chip_20250811-133822.npy", 
#                           r"\multi_transmon_chip_20250811-152215.npy", 
#                           r"\multi_transmon_chip_20250811-170027.npy"] 
# 10_4 wide initials
# child_path_initials = r"\single_qubit_resonator_fullchi_10_4_wideinitials"
# f_sim_result_variation = [r"\multi_transmon_chip_20250813-151950.npy", 
#                           r"\multi_transmon_chip_20250813-172151.npy", 
#                           r"\multi_transmon_chip_20250813-224452.npy", 
#                           r"\multi_transmon_chip_20250814-063943.npy", 
#                           r"\multi_transmon_chip_20250814-151031.npy",
#                           r"\multi_transmon_chip_20250814-155853.npy", 
#                           r"\multi_transmon_chip_20250814-194811.npy",
#                           r"\multi_transmon_chip_20250814-204059.npy", 
#                           r"\multi_transmon_chip_20250814-213905.npy",
#                           r"\multi_transmon_chip_20250815-090407.npy"] 

# 10_4_wide_full chi
child_path_initials = r"\single_qubit_resonator_detailedchi_10_4_wideinitials"
f_sim_result_variation = [r"\multi_transmon_chip_20250822-105223.npy", 
                          r"\multi_transmon_chip_20250822-125214.npy", 
                          r"\multi_transmon_chip_20250822-133856.npy", 
                          r"\multi_transmon_chip_20250822-142604.npy", 
                          r'\multi_transmon_chip_20250822-152333.npy',
                          r"\multi_transmon_chip_20250822-162827.npy",
                          r"\multi_transmon_chip_20250822-175650.npy", 
                          r"\multi_transmon_chip_20250822-214021.npy",
                          r"\multi_transmon_chip_20250823-080244.npy", 
                          r"\multi_transmon_chip_20250823-123457.npy",
                          ] 


In [ ]:
opt_plot_variations = []
for i in range(len(f_sim_result_variation)):
    d_sim_result = np.load(parent_path + child_path_initials + f_sim_result_variation[i], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params)
    plotter = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)
    opt_plot_variations.append(plotter)

### Plot figure with data

In [ ]:
figure_width_inch = 3.35
figure_height_inch = 3.5
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, figure_height_inch))
gs = fig.add_gridspec(2, 2, wspace=0.45, hspace=0.4, left=0.12, right=0.98, bottom=0.13, top=0.97)
ax0 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

# comparison of nonlinear models
opt_plot_Fig4[0]._plot_single_param(ax0, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['blue'], markersize=5)
opt_plot_Fig4[1]._plot_single_param(ax0, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['orange'], markersize=5)
opt_plot_Fig4[2]._plot_single_param(ax0, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['grey'], markersize=5, alpha=0.15) # sqrt
opt_plot_Fig4[3]._plot_single_param(ax0, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['purple'], markersize=5) # x2
opt_plot_Fig4[5]._plot_single_param(ax0, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['green'], markersize=5) # sqrt x adj 0.5
opt_plot_Fig4[6]._plot_single_param(ax0, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['pink'], markersize=5) # x3
opt_plot_Fig4[1]._plot_target(ax0, config['RES_QUB_chi'].y, 0, 15, colors_got['grey'], config['RES_QUB_chi'].normalization)

ax0.set_xticks([5,10])
ax0.set_xlabel('Iteration')
ax0.text(0.1,0.6,r'$\chi \sim (w/w_{qb})^2 \alpha / [\Delta (\Delta - \alpha) ] $', transform=ax0.transAxes, color=colors_got['blue'])
ax0.text(0.4,0.45,'$\chi \sim w$', transform=ax0.transAxes, color=colors_got['orange'])
ax0.text(0.5,0.05,'$\chi \sim w^2$', transform=ax0.transAxes, color=colors_got['purple'])
ax0.text(0.7,0.1,'$\chi \sim w^3$', transform=ax0.transAxes, color=colors_got['pink'])
ax0.text(0.65,0.75,'$\chi \sim \sqrt{w}$', transform=ax0.transAxes, color=colors_got['grey'])
ax0.text(0.6,0.4,'$\chi \sim \sqrt{w}, \gamma=0.5$', transform=ax0.transAxes, color=colors_got['green'])

# Comparison of different initial conditions
for i in [0,6]: # selection of initial conditions showing no good optimization
    # opt_plot_variations[i]._plot_single_param(ax3, config['RES_freq'], y_param=config['RES_freq'].y, color=colors_got['grey'], markersize=5, alpha=0.15)
    opt_plot_variations[i]._plot_single_param(ax2, config['QUB_freq'], y_param=config['QUB_freq'].y, color=colors_got['grey'], markersize=5, alpha=0.15)
    opt_plot_variations[i]._plot_single_param(ax3, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['grey'], markersize=5, alpha=0.15)
    

n_variations = len(f_sim_result_variation)
# for i in [3]: # selection of initial conditions showing good optimization
for i in [1,2,3,4,5,7]: # selection of initial conditions showing good optimization
    # opt_plot_variations[i]._plot_single_param(ax1, config['RES_freq'], y_param=config['RES_freq'].y, color=color_shades_blue[i], markersize=5)
    opt_plot_variations[i]._plot_single_param(ax2, config['QUB_freq'], y_param=config['QUB_freq'].y, color=color_shades_blue[i], markersize=5)
    opt_plot_variations[i]._plot_single_param(ax3, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=color_shades_blue[i], markersize=5)


# ax1.set_xticks([5,10])
ax2.set_xticks([5,10])
ax3.set_xticks([5,10])
# ax1.set_xlabel('Iteration')
ax2.set_xlabel('Iteration')
ax3.set_xlabel('Iteration')

ax2.set_ylim(2.5,5.8)
ax3.set_ylim(0,5)

ax0.text(0.025,0.88,'(a)', transform=ax0.transAxes)
ax2.text(0.1,0.88,'(b)', transform=ax2.transAxes)
ax3.text(0.3,0.88,'(c)', transform=ax3.transAxes)

fig.savefig(parent_path + r"\Fig3_COMP_single_qubit_resonator.png", dpi=300)

## Fig 5: 2QB-CPLR chip

In [ ]:
child_path_2qb = r'\two_qubit_resonator_5modes'

### Load images

In [ ]:
cropped_image_2qb = crop_image_by_fraction(parent_path + child_path_2qb + r'\2qb_cplr_medqual_fc_sb.png', left=0.13, right=0.13, top=0.2, bottom=0.27, rotate_90=0)

### Load data

In [ ]:
# f_sim_result_2qb = [parent_path  + child_path_2qb + r'\multi_transmon_chip_20250822-085742.npy'] # 7 iterations
f_sim_result_2qb = [parent_path  + child_path_2qb + r'\multi_transmon_chip_20250823-103823.npy']

In [ ]:
d_sim_result = np.load(f_sim_result_2qb[0], allow_pickle=True)[0]
system_target_params = d_sim_result["system_target_params"]
opt_result = d_sim_result['optimization_results']

opt_target_list = ot.get_opt_targets_2qubits_resonator_coupler([1], True,True,True,True,True,False,False)

iteration = 'Iteration'
config = {
    'RES1_freq': OptPltSet(n.ITERATION, param(n.RESONATOR_1, n.FREQ), y_label="RES Freq", unit="GHz"), 
    'RES2_freq': OptPltSet(n.ITERATION, param(n.RESONATOR_2, n.FREQ), y_label="RES Freq", unit="GHz"), 
    'QUB1_freq': OptPltSet(n.ITERATION, param(n.QUBIT_1, n.FREQ), y_label="QB Freq", unit="GHz"),
    'QUB2_freq': OptPltSet(n.ITERATION, param(n.QUBIT_2, n.FREQ), y_label="$f_{qb}$", unit="GHz"),
    'CPLR_freq': OptPltSet(n.ITERATION, param('coupler_1to2', n.FREQ), y_label="$f$", unit="GHz"),
    'QUB1_anh': OptPltSet(n.ITERATION, param_nonlin(n.QUBIT_1, n.QUBIT_1), y_label="QB Anharm.", unit="MHz"),
    'QUB2_anh': OptPltSet(n.ITERATION, param_nonlin(n.QUBIT_2, n.QUBIT_2), y_label=r"$\alpha_{qb}$", unit="MHz"),
    'CPLR_anh': OptPltSet(n.ITERATION, param_nonlin('coupler_1to2', 'coupler_1to2'), y_label=r"$\alpha$", unit="MHz"),
    'RES1_QUB1_chi': OptPltSet(n.ITERATION, param_nonlin(n.RESONATOR_1, n.QUBIT_1), y_label="RES-QB Chi", unit="MHz"),
    'RES2_QUB2_chi': OptPltSet(n.ITERATION, param_nonlin(n.RESONATOR_2, n.QUBIT_2), y_label=r"$\chi_{qb-res}$", unit="MHz"),
    'QUB1_CPLR_chi': OptPltSet(n.ITERATION, param_nonlin(n.QUBIT_1, 'coupler_1to2'), y_label="QB-CPLR Chi", unit="MHz"),
    'QUB2_CPLR_chi': OptPltSet(n.ITERATION, param_nonlin(n.QUBIT_2, 'coupler_1to2'), y_label=r"$\chi_{qb-cplr}$", unit="MHz"),
    }

print(system_target_params)
print(opt_result[0]['design_variables'])


In [ ]:
data_extractor_2qb = DataExtractor([opt_result], system_target_params, opt_target_list)
plotter_2qb = OptimizationPlotter(data_extractor_2qb, plot_variance=False, save_figures=False)

In [ ]:
figure_width_inch = 3.35
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, 4.5))
gs = fig.add_gridspec(3, 2, wspace=0.55, hspace=0.15, left=0.12, right=0.96, bottom=0.10, top=0.98)
ax0 = fig.add_subplot(gs[0, :])

ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[2, 0])
ax5 = fig.add_subplot(gs[2, 1])

# image
ax0.imshow(cropped_image_2qb)
ax0.axis('off')

# Modes 
## Resonator
plotter_2qb._plot_single_param(ax2, config['RES1_freq'], y_param=config['RES1_freq'].y, color=colors_got['blue'], markersize=5, marker='v')
plotter_2qb._plot_single_param(ax2, config['RES2_freq'], y_param=config['RES2_freq'].y, color=colors_got['orange'], markersize=5, marker='v')
plotter_2qb._plot_single_param(ax2, config['QUB1_freq'], y_param=config['QUB1_freq'].y, color=colors_got['blue'], markersize=5)
plotter_2qb._plot_single_param(ax2, config['QUB2_freq'], y_param=config['QUB2_freq'].y, color=colors_got['orange'], markersize=5)
plotter_2qb._plot_single_param(ax2, config['CPLR_freq'], y_param=config['CPLR_freq'].y, color=colors_got['green'], markersize=5)

# Nonlinearities
## Anharmonicity
plotter_2qb._plot_single_param(ax3, config['QUB1_anh'], y_param=config['QUB1_anh'].y, color=colors_got['blue'], markersize=5)
plotter_2qb._plot_single_param(ax3, config['QUB2_anh'], y_param=config['QUB2_anh'].y, color=colors_got['orange'], markersize=5)
plotter_2qb._plot_single_param(ax3, config['CPLR_anh'], y_param=config['CPLR_anh'].y, color=colors_got['green'], markersize=5)
## RES-QB chi
plotter_2qb._plot_single_param(ax5, config['RES1_QUB1_chi'], y_param=config['RES1_QUB1_chi'].y, color=colors_got['blue'], markersize=5)
plotter_2qb._plot_single_param(ax5, config['RES2_QUB2_chi'], y_param=config['RES2_QUB2_chi'].y, color=colors_got['orange'], markersize=5)
## QB-CPLR chi
plotter_2qb._plot_single_param(ax4, config['QUB1_CPLR_chi'], y_param=config['QUB1_CPLR_chi'].y, color=colors_got['blue'], markersize=5)
plotter_2qb._plot_single_param(ax4, config['QUB2_CPLR_chi'], y_param=config['QUB2_CPLR_chi'].y, color=colors_got['orange'], markersize=5)

ax2.set_xticks([5,10])
ax3.set_xticks([5,10])
ax4.set_xticks([5,10])
ax5.set_xticks([5,10])

ax2.set_xlabel('')
ax3.set_xlabel('')
ax4.set_xlabel('Iteration')
ax5.set_xlabel('Iteration')

ax2.set_xticklabels([])
ax3.set_xticklabels([])

ax0.text(0.025,0.85,'(a)', transform=ax0.transAxes)
ax2.text(0.1,0.75,'(b)', transform=ax2.transAxes)
ax3.text(0.1,0.7,'(c)', transform=ax3.transAxes)
ax4.text(0.1,0.1,'(d)', transform=ax4.transAxes)
ax5.text(0.2,0.88,'(e)', transform=ax5.transAxes)

ax2.text(0.25,0.75,'RES1', transform=ax2.transAxes, color=colors_got['blue'])
ax2.text(0.3,0.2,'QB1', transform=ax2.transAxes, color=colors_got['blue'])
ax2.text(0.65,0.75,'RES2', transform=ax2.transAxes, color=colors_got['orange'])
ax2.text(0.6,0.2,'QB2', transform=ax2.transAxes, color=colors_got['orange'])
ax2.text(0.4,0.55,'CPLR', transform=ax2.transAxes, color=colors_got['green'])

fig.savefig(parent_path + r"\Fig4_OPT_two_qubit_resonator.png", dpi=300)

# Figures for supplement

## Fig S1: Capacitance targets

### Load images

In [ ]:
child_path_capacitance = r'\capacitance_resonator_feedline'
child_path_decay = r"\chargeline_qubit"

In [ ]:
cropped_image_capacitance = crop_image_by_fraction(parent_path + child_path_capacitance + r'\2025-08-10_OPT_resonator_feedline_capacitance_final_zoom_2.png', left=0.1, right=0.1, top=0.0, bottom=0.1, rotate_90=0)
cropped_image_decay = crop_image_by_fraction(parent_path + child_path_decay + r'\2025-08-10_OPT_chargeline_final_zoom.png', left=0., right=0.0, top=0.2, bottom=0.0, rotate_90=0)

### Load data

In [ ]:
f_sim_result = [parent_path  + child_path_capacitance + r'\multi_transmon_chip_20250811-104854.npy', parent_path + child_path_decay + r'\multi_transmon_chip_20250811-105908.npy']

In [ ]:
opt_plot = []
for i in range(len(f_sim_result)):
    d_sim_result = np.load(f_sim_result[i], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params)
    plotter = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)
    opt_plot.append(plotter)

In [ ]:
iteration = 'Iteration'
config = {
    'res_fd_coupling': OptPltSet(n.ITERATION, param_capacitance("prime_cpw_name_tee1_", "second_cpw_name_tee1_"), y_label="Capacitance", unit="fF"), 
    'charge_line': OptPltSet(n.ITERATION, param(n.QUBIT_1, n.CHARGE_LINE_LIMITED_T1), y_label="T1 limit", y_scale="linear", unit="ms")
    }

### Plot figure with data and images

In [ ]:
figure_width_inch = 3.35
figure_height_inch = 3.2
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, figure_height_inch))
gs = fig.add_gridspec(2, 2, wspace=0.7, hspace=0.4, left=0.16, right=0.96, bottom=0.14, top=0.95)
ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[0,1])
ax4 = fig.add_subplot(gs[1,1])

ax1.imshow(cropped_image_capacitance)
ax2.imshow(cropped_image_decay)

ax1.axis('off')
ax2.axis('off')


opt_plot[0]._plot_single_param(ax3, config['res_fd_coupling'], y_param=config['res_fd_coupling'].y, color=colors_got['blue'], markersize=5)
opt_plot[1]._plot_single_param(ax4, config['charge_line'], y_param=config['charge_line'].y, color=colors_got['blue'], markersize=5)

ax3.set_xticks([5,10])
ax4.set_xticks([5,10])
ax3.set_ylim([-4.5,-2.3])
ax4.set_ylim([0,25])

ax3.set_xlabel('Iteration')
ax4.set_xlabel('Iteration')

ax1.text(-0.3,0.88,'(a)', transform=ax1.transAxes)
ax2.text(-0.3,0.88,'(b)', transform=ax2.transAxes)
ax3.text(0.1,0.88,'(c)', transform=ax3.transAxes)
ax4.text(0.1,0.88,'(d)', transform=ax4.transAxes)

fig.savefig(parent_path + r"\FigS1_OPT_capacitance_decay.png", dpi=300)

## S2: Effect of simulation parameters on optimization landscape

### Load data

In [ ]:
child_path_passes = r"\single_qubit_resonator_fullchi_10_passes"
f_sim_result = [r'\multi_transmon_chip_20250818-104818.npy', # 3
                r'\multi_transmon_chip_20250815-095948.npy', # 4
                 r'\multi_transmon_chip_20250815-110158.npy', # 5
                 r'\multi_transmon_chip_20250815-130510.npy', # 6
                 r'\multi_transmon_chip_20250815-144537.npy', # 7
                 r'\multi_transmon_chip_20250810-175429.npy', # 8
                 r'\multi_transmon_chip_20250817-140710.npy', # 9
                 r'\multi_transmon_chip_20250816-070833.npy', # 10
                 ]

opt_plot = []
for i in range(len(f_sim_result)):
    d_sim_result = np.load(parent_path  + child_path_passes + f_sim_result[i], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params)
    plotter = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)
    opt_plot.append(plotter)

In [ ]:
config = {
    'RES_freq': OptPltSet(n.ITERATION, param(n.RESONATOR_1, n.FREQ), y_label="RES Freq", unit="GHz"), 
    'RES_kappa': OptPltSet(n.ITERATION, param(n.RESONATOR_1, n.KAPPA), y_label="$\kappa_{res}$", unit="MHz"),
    'QUB_freq': OptPltSet(n.ITERATION, param(n.QUBIT_1, n.FREQ), y_label="QB Freq", unit="GHz"),
    'QUB_anh': OptPltSet(n.ITERATION, param_nonlin(n.QUBIT_1, n.QUBIT_1), y_label="QB Anharm.", unit="MHz"),
    'RES_QUB_chi': OptPltSet(n.ITERATION, param_nonlin(n.RESONATOR_1, n.QUBIT_1), y_label="RES-QB Chi", unit="MHz"),
    }

### Optimization Targets

In [ ]:
figure_width_inch = 7
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, 3.5))
gs = fig.add_gridspec(2, 3, wspace=0.55, hspace=0.15, left=0.1, right=0.96, bottom=0.13, top=0.95)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1])
ax5 = fig.add_subplot(gs[1, 2])
for i in range(len(f_sim_result)):
    opt_plot[i]._plot_single_param(ax1, config['RES_freq'], y_param=config['RES_freq'].y, color=color_shades_blue[i], markersize=5)
    opt_plot[i]._plot_single_param(ax2, config['RES_kappa'], y_param=config['RES_kappa'].y, color=color_shades_blue[i], markersize=5)
    opt_plot[i]._plot_single_param(ax3, config['QUB_freq'], y_param=config['QUB_freq'].y, color=color_shades_blue[i], markersize=5)
    opt_plot[i]._plot_single_param(ax4, config['QUB_anh'], y_param=config['QUB_anh'].y, color=color_shades_blue[i], markersize=5)
    opt_plot[i]._plot_single_param(ax5, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=color_shades_blue[i], markersize=5)

ax1.set_xticks([5,10])
ax2.set_xticks([5,10])
ax3.set_xticks([5,10])
ax4.set_xticks([5,10])
ax5.set_xticks([5,10])

ax1.set_xlabel('')
ax2.set_xlabel('')
ax3.set_xlabel('Iteration')
ax4.set_xlabel('Iteration')
ax5.set_xlabel('Iteration')

ax1.set_xticklabels([])
ax2.set_xticklabels([])

ax1.text(0.05,0.1,'(a)', transform=ax0.transAxes)
ax1.text(0.1,0.88,'(b)', transform=ax1.transAxes)
ax2.text(0.04,0.88,'(c)', transform=ax2.transAxes)
ax3.text(0.1,0.88,'(d)', transform=ax3.transAxes)
ax4.text(0.1,0.88,'(e)', transform=ax4.transAxes)
ax5.text(0.1,0.88,'(f)', transform=ax5.transAxes)

ax0.imshow(cropped_image)
ax0.axis('off')

fig.savefig(parent_path + r"\FigS2_OPT_single_qubit_resonator_passes.png", dpi=300)

### Design variables

In [ ]:
data_vs_passes = np.zeros((len(f_sim_result), 5))  # 5 design variables
for i in range(len(f_sim_result)):
    d_sim_result = np.load(parent_path  + child_path_passes + f_sim_result[i], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params)

    
    for k, des_var in enumerate(["design_var_lj_qubit_1", "design_var_width_qubit_1", "design_var_length_resonator_1", "design_var_coupl_length_resonator_1_tee", "design_var_coupl_length_qubit_1_resonator_1"]):
        data_vs_passes[i, k] = data_extractor.get_parameter_value(des_var, result=opt_result[-1], iteration=0)

list_of_passes = np.array([3,4,5,6,7,8,9,10])


In [ ]:
figure_width_inch = 3.35
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, 3.2))
gs = fig.add_gridspec(2, 1, wspace=0.55, hspace=0.4, left=0.15, right=0.97, bottom=0.13, top=0.96)
ax1 = fig.add_subplot(gs[0, 0])
ax0 = fig.add_subplot(gs[1, 0])
for i in range(len(f_sim_result)):
    opt_plot[i]._plot_single_param(ax0, config['RES_kappa'], y_param=config['RES_kappa'].y, color=color_shades_lightblue[i], markersize=5)

ax0.text(0.4,0.5,'$3~passes$', transform=ax0.transAxes, color=color_shades_lightblue[0])
ax0.text(0.7,0.5,'$10~passes$', transform=ax0.transAxes, color=color_shades_lightblue[i])

ax1.plot(list_of_passes, data_vs_passes[:, 0]/data_vs_passes[-1, 0], marker='o', color=colors_got['purple'], label='LJ Qubit 1')
ax1.plot(list_of_passes, data_vs_passes[:, 1]/data_vs_passes[-1, 1], marker='o', color=colors_got['pink'], label='Width Qubit 1')
ax1.plot(list_of_passes, data_vs_passes[:, 2]/data_vs_passes[-1, 2], marker='o', color=colors_got['blue'], label='Length Resonator 1')
ax1.plot(list_of_passes, data_vs_passes[:, 3]/data_vs_passes[-1, 3], marker='o', color=colors_got['lightblue'], label='Coupling Length Resonator 1 Tee')
ax1.plot(list_of_passes, data_vs_passes[:, 4]/data_vs_passes[-1, 4], marker='o', color=colors_got['orange'], label='Coupling Length Qubit 1 Resonator 1')

ax0.set_xlabel('Iteration')

ax1.set_xlabel('Passes')
ax1.set_ylabel('Norm. Design Var.')

ax0.text(0.05,0.88,'(b)', transform=ax0.transAxes)
ax1.text(0.05,0.88,'(a)', transform=ax1.transAxes)

ax1.text(0.05,0.55,'$l_{res}$', transform=ax1.transAxes, color=colors_got['blue'])
ax1.text(0.15,0.5,'$L_{qb}$', transform=ax1.transAxes, color=colors_got['purple'])
ax1.text(0.25,0.05,'$w_{qb}$', transform=ax1.transAxes, color=colors_got['pink'])
ax1.text(0.4,0.15,'$w_{res-qb}$', transform=ax1.transAxes, color=colors_got['orange'])
ax1.text(0.75,0.7,'$l_{res-tl}$', transform=ax1.transAxes, color=colors_got['lightblue'])

fig.savefig(parent_path + r"\FigS2_OPT_single_qubit_resonator_passes_des_var.png", dpi=300)

## S3: Tracking design variable updates

In [ ]:
des_var_vs_iteration = np.zeros((10, 10, 5))  # 10 opts, 10 iterations, 5 design variables
for i, run in enumerate([1,2,3,4,5,7,8,9]):
    d_sim_result = np.load(parent_path  + child_path_initials + f_sim_result_variation[run], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params)

    for j in range(10):
        for k, des_var in enumerate(["design_var_lj_qubit_1", "design_var_width_qubit_1", "design_var_length_resonator_1", "design_var_coupl_length_resonator_1_tee", "design_var_coupl_length_qubit_1_resonator_1"]):
            des_var_vs_iteration[i, j, k] = data_extractor.get_parameter_value(des_var, result=opt_result[j], iteration=0)

iterations = np.arange(start=1, stop=11, step=1)


In [ ]:
figure_width_inch = 3.35
figure_height_inch = 3.2
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, figure_height_inch))
gs = fig.add_gridspec(2, 2, wspace=0.7, hspace=0.4, left=0.16, right=0.98, bottom=0.08, top=0.98)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,:])

# tracking of two design variables: qubit Lj
ax0.plot(iterations, des_var_vs_iteration[0,:,0], label='Design Var 1', color=color_shades_purple[0], marker='o', ms=5)
ax0.plot(iterations, des_var_vs_iteration[1,:,0], label='Design Var 1', color=color_shades_purple[5], marker='o', ms=5)
# tracking of two design variables: resonator length
ax1.plot(iterations, des_var_vs_iteration[0,:,2]/1e3, label='Design Var 2', color=color_shades_blue[0], marker='o', ms=5)
ax1.plot(iterations, des_var_vs_iteration[1,:,2]/1e3, label='Design Var 2', color=color_shades_blue[5], marker='o', ms=5)

# tracking of chi
opt_plot_Fig4[0]._plot_param_vs_design_var(ax2, config_Fig3['RES_QUB_chi'], y_param=config_Fig3['RES_QUB_chi'].y, color=colors_got['blue'], sort_by_x=False, markersize=5)
opt_plot_Fig4[1]._plot_param_vs_design_var(ax2, config_Fig3['RES_QUB_chi'], y_param=config_Fig3['RES_QUB_chi'].y, color=colors_got['orange'], sort_by_x=False, markersize=5)
opt_plot_Fig4[2]._plot_param_vs_design_var(ax2, config_Fig3['RES_QUB_chi'], y_param=config_Fig3['RES_QUB_chi'].y, color=colors_got['grey'], sort_by_x=False, markersize=5, alpha=0.15)
opt_plot_Fig4[3]._plot_param_vs_design_var(ax2, config_Fig3['RES_QUB_chi'], y_param=config_Fig3['RES_QUB_chi'].y, color=colors_got['purple'], sort_by_x=False, markersize=5)
opt_plot_Fig4[5]._plot_param_vs_design_var(ax2, config_Fig3['RES_QUB_chi'], y_param=config_Fig3['RES_QUB_chi'].y, color=colors_got['green'], sort_by_x=False, markersize=5)
opt_plot_Fig4[6]._plot_param_vs_design_var(ax2, config_Fig3['RES_QUB_chi'], y_param=config_Fig3['RES_QUB_chi'].y, color=colors_got['pink'], sort_by_x=False, markersize=5)
opt_plot_Fig4[1]._plot_target(ax2, config_Fig3['RES_QUB_chi'].y, 0, 15, colors_got['grey'], config_Fig3['RES_QUB_chi'].normalization)

ax0.set_xticks([5,10])
ax1.set_xticks([5,10])
ax2.set_xticks([5,10])
ax0.set_xlabel('Iteration')
ax1.set_xlabel('Iteration')
ax0.set_ylabel('Qubit $L_j$ (nH)')
ax1.set_ylabel('RES length (mm)')
ax2.set_xlabel('Coupling length QUB-RES $w_{res-qb}$ (mm)')

ax2.set_ylim(0.2,1.9)
ax2.set_xlim(20,500)

ax0.text(0.2,0.88,'(a)', transform=ax0.transAxes)
ax1.text(0.1,0.88,'(b)', transform=ax1.transAxes)
ax2.text(0.05,0.88,'(c)', transform=ax2.transAxes)


fig.savefig(parent_path + r"\FigS3_OPT_single_qubit_resonator_tracking_des_var.png", dpi=300)

## S5: Parameter sweep

In [ ]:
child_path_parameter = r"\single_qubit_resonator_parametersweep"
f_sim_result = [r'\multi_transmon_chip_20250915-125727.npy']
opt_plot = []

d_sim_result = np.load(parent_path  + child_path_parameter + f_sim_result[0], allow_pickle=True)[0]
system_target_params = d_sim_result["system_target_params"]
opt_result = d_sim_result['optimization_results']

data_extractor = DataExtractor([opt_result], system_target_params)

In [ ]:
data_parameter_sweep = np.zeros((10, 2))  # 10 iterations, 5 design variables
for i in range(10):
    data_parameter_sweep[i, 0] = data_extractor.get_parameter_value("design_var_coupl_length_qubit_1_resonator_1", result=opt_result[i], iteration=0)
    data_parameter_sweep[i, 1] = data_extractor.opt_results[0][i]['system_optimized_params']['qubit_1_to_resonator_1_nonlin']
    

In [ ]:
def powerlaw(x, a, b):
    return a * np.power(x, b)

In [ ]:
figure_width_inch = 3.35
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, 1.6))
gs = fig.add_gridspec(1, 1, wspace=0.55, hspace=0.15, left=0.16, right=0.99, bottom=0.28, top=0.94)
ax0 = fig.add_subplot(gs[0, 0])

ax0.plot(data_parameter_sweep[:,0], data_parameter_sweep[:,1]/1e6, marker='o', color=colors_got['blue'], markersize=5)
x = np.linspace(7,700,100)
ax0.plot(x, powerlaw(x,a=0.012, b=1), linestyle='--', color=colors_got['grey'])

ax0.set_xlabel('Coupl. length QUB-RES (nm)')
ax0.set_ylabel(r"$\chi_{qb-cplr}$ (MHz)")
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_ylim(0.4,10)

ax0.text(0.65,0.15,r'$\chi \sim a w_{res-qb}$', transform=ax0.transAxes, color=colors_got['grey'])

fig.savefig(parent_path + r"\FigS5_OPT_single_qubit_resonator_parametersweep.png", dpi=300)

# QR Codes

In [ ]:
# pip install qrcode

In [ ]:
import qrcode
qr = qrcode.QRCode(version=3, box_size=15, border=5, error_correction=qrcode.constants.ERROR_CORRECT_L)
data = "https://github.com/202Q-lab/QDesignOptimizer"
qr.add_data(data)
qr.make(fit=True)
img = qr.make_image(fill_color="black", back_color="white")
img.save("git_qr_code.png")

In [ ]:
qr = qrcode.QRCode(version=3, box_size=15, border=5, error_correction=qrcode.constants.ERROR_CORRECT_L)
data = "https://arxiv.org/abs/2508.18027"
qr.add_data(data)
qr.make(fit=True)
img = qr.make_image(fill_color="black", back_color="white")
img.save("paper_qr_code.png")